In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from datetime import datetime
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = tf.image.resize(train_images[..., tf.newaxis], (32, 32))
train_images = np.repeat(train_images, 3, axis=-1)
test_images = tf.image.resize(test_images[..., tf.newaxis], (32, 32))
test_images = np.repeat(test_images, 3, axis=-1)
train_images = train_images / 255.0
test_images = test_images/ 255.0

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)


4422102/4422102 [==============================] - 1s 0us/step


**Побудова СNN на принципі виділення ознак.**

*Так як модель потрібна більше для порівняння, немає сенсу брати велику кількість епох, так як це значно збільшує час навчання.*

In [ ]:
conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
conv_base.trainable = False

model = Sequential([
   conv_base,
   layers.Flatten(),
   layers.Dense(256, activation="relu"),
   layers.Dense(10, activation="softmax"),
])

model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
    metrics=["acc"]
)

history = model.fit(train_images, train_labels, epochs=5, verbose=1, batch_size=64, validation_split=0.2)

Epoch 1/3
750/750 [==============================] - 703s 936ms/step - loss: 0.5670 - acc: 0.8014 - val_loss: 0.4662 - val_acc: 0.8318
Epoch 2/3
750/750 [==============================] - 634s 846ms/step - loss: 0.4209 - acc: 0.8466 - val_loss: 0.4256 - val_acc: 0.8447
Epoch 3/3
750/750 [==============================] - 636s 848ms/step - loss: 0.3836 - acc: 0.8591 - val_loss: 0.4198 - val_acc: 0.8500


In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

313/313 [==============================] - 106s 338ms/step - loss: 0.4243 - acc: 0.8489
Test loss: 0.4243021607398987
Test accuracy: 0.8489000201225281


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

Використання принципу донавчання з розморожуванням трьох верхніх шарів. Усі параметри залишаються як і в попередній моделі, лише додається донавчання трьох шарів.

In [ ]:
conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == "block5_conv1":
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

modified_model = Sequential([
   conv_base,
   layers.Flatten(),
   layers.Dense(256, activation="relu"),
   layers.Dense(10, activation="softmax"),
])

modified_model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
    metrics=["acc"]
)

history = modified_model.fit(train_images, train_labels, epochs=3, verbose=1, batch_size=64, validation_split=0.2)

58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/3
750/750 [==============================] - 1966s 3s/step - loss: 0.4476 - acc: 0.8409 - val_loss: 0.3646 - val_acc: 0.8703
Epoch 2/3
750/750 [==============================] - 1973s 3s/step - loss: 0.3181 - acc: 0.8876 - val_loss: 0.3052 - val_acc: 0.8883
Epoch 3/3
750/750 [==============================] - 1932s 3s/step - loss: 0.2782 - acc: 0.8987 - val_loss: 0.2934 - val_acc: 0.8982


Використовуючи донавчання вийшло досягнути точності, кращої за багатошарову nn, але аналізуючи результат згорткової nnб можна зробити висновок, що для таких не дуже складних класифікацій краще писати архітектуру з нуля ніж засновувати її на таких складних алгоритмах